# Initial data exploration: data cleaning

In [1]:
import numpy as np
import pandas as pd

In [2]:
def basic_analysis(DF):
    print(DF.shape)
    basic_info = pd.DataFrame(DF.isna().sum(),columns=['# NULL'])
    basic_info['Type'] = DF.dtypes.values
    basic_info['Unique'] = DF.nunique()
    return basic_info

In [3]:
steam = pd.read_csv('data/steam.csv')

### Start by cleaning the steam dataset

In [4]:
bi = basic_analysis(steam)
bi

(27075, 18)


,# NULL,Type,Unique
appid,0,int64,27075
name,0,object,27033
release_date,0,object,2619
english,0,int64,2
developer,0,object,17113
publisher,0,object,14354
platforms,0,object,7
required_age,0,int64,6
categories,0,object,3333
genres,0,object,1552


In [5]:
steam.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [6]:
# First make owners an integer
def make_owners_int(x):
    numbers = x.split('-')
    numbers = [int(i) for i in numbers]
    return round((numbers[1]+numbers[0])/2)

steamnew = steam.copy()
steamnew['num_owners'] = steamnew.owners.apply(make_owners_int)

In [7]:
steamnew.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,num_owners
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,15000000
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,7500000
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,7500000
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,7500000
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,7500000


In [8]:
# Now, how many unique values in genres?
steam.genres.unique()

array(['Action', 'Action;Free to Play', 'Action;Free to Play;Strategy',
       ...,
       'Action;Adventure;Indie;Massively Multiplayer;RPG;Strategy;Early Access',
       'Action;Adventure;Casual;Free to Play;Indie;RPG;Simulation;Sports;Strategy',
       'Casual;Free to Play;Massively Multiplayer;RPG;Early Access'],
      dtype=object)

In [9]:
steam_wide = steamnew.copy() # Wide format with each game having one row.

### Idea: Separate the multiple genres into new rows; one per genre per game. There will be multiple rows per game, but then we can aggregate by genre because each row will only have one genre.  

In [10]:
steamnew.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price', 'num_owners'],
      dtype='object')

In [11]:
# convert to list
steamnew['genres'] = steamnew['genres'].str.split(';')

# convert list of pd.Series then stack it
steamnew = (steamnew
 .set_index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price', 'num_owners'])['genres']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .drop('level_18', axis=1)
 .rename(columns={0:'genre'}))

In [12]:
steamnew

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,num_owners,genre
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,15000000,Action
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,7500000,Action
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,7500000,Action
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,7500000,Action
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,7500000,Action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76457,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19,10000,Casual
76458,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19,10000,Indie
76459,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Indie;Casual;Adventure,0,4,0,0,0,0-20000,5.19,10000,Adventure
76460,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Indie;Casual;Adventure,0,4,0,0,0,0-20000,5.19,10000,Casual


In [13]:
steam_long = steamnew.copy() # Long format is really only long in genre, but still.

### Join the description dataset to the wide dataset.

In [14]:
steamnew = steam_wide.copy()
desc = pd.read_csv('data/steam_description_data.csv')

In [15]:
bi = basic_analysis(steam_wide)
bi

(27075, 19)


,# NULL,Type,Unique
appid,0,int64,27075
name,0,object,27033
release_date,0,object,2619
english,0,int64,2
developer,0,object,17113
publisher,0,object,14354
platforms,0,object,7
required_age,0,int64,6
categories,0,object,3333
genres,0,object,1552


In [16]:
bi2 = basic_analysis(desc)
bi2

(27334, 4)


,# NULL,Type,Unique
steam_appid,0,int64,27334
detailed_description,0,object,27315
about_the_game,0,object,27315
short_description,0,object,27204


In [17]:
# Update the column names of desc.
desc = desc.rename(columns={'steam_appid':"appid"})
desc.columns

Index(['appid', 'detailed_description', 'about_the_game', 'short_description'], dtype='object')

In [18]:
steamnew = steamnew.set_index('appid').join(other=desc.set_index('appid'),on='appid',how='left',rsuffix='_desc')
steamnew.shape
steamnew = steamnew.reset_index()

In [19]:
steam_wide = steamnew.copy()

In [20]:
steam_wide.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,num_owners,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,124534,3339,17612,317,10000000-20000000,7.19,15000000,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,3318,633,277,62,5000000-10000000,3.99,7500000,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,3416,398,187,34,5000000-10000000,3.99,7500000,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,1273,267,258,184,5000000-10000000,3.99,7500000,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,5250,288,624,415,5000000-10000000,3.99,7500000,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


### Prep data to rewrite to csv

In [21]:
steam_long_columns = ['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price', 'num_owners',
       'genre']

new_steam_long_columns = ['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'price', 'num_owners',
       'genre']

steam_wide_columns = ['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price', 'num_owners',
       'detailed_description', 'about_the_game', 'short_description']

new_steam_wide_columns = ['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'price', 'num_owners',
       'detailed_description', 'about_the_game', 'short_description']

In [22]:
steam_long_out = steam_long[new_steam_long_columns].copy()
steam_long_out.tail()

,appid,name,release_date,english,developer,publisher,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,num_owners,genre
76457,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,0,2,0,0,0,5.19,10000,Casual
76458,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,0,2,0,0,0,5.19,10000,Indie
76459,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,0,4,0,0,0,5.19,10000,Adventure
76460,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,0,4,0,0,0,5.19,10000,Casual
76461,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,0,4,0,0,0,5.19,10000,Indie


In [23]:
steam_wide_out = steam_wide[new_steam_wide_columns].copy()
steam_wide_out.tail()

,appid,name,release_date,english,developer,publisher,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,num_owners,detailed_description,about_the_game,short_description
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,7,3,0,0,0,2.09,10000,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",The Room of Pandora is a third-person interact...
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,0,8,1,0,0,1.69,10000,Have you ever been so lonely that no one but y...,Have you ever been so lonely that no one but y...,Cyber Gun is a hardcore first-person shooter w...
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,24,0,1,0,0,3.99,10000,<strong>Super Star Blast </strong>is a space b...,<strong>Super Star Blast </strong>is a space b...,Super Star Blast is a space based game with ch...
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,0,2,0,0,0,5.19,10000,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...
27074,1069460,Rune Lord,2019-04-24,1,Adept Studios GD,Alawar Entertainment,0,4,0,0,0,5.19,10000,A portal has opened and dark magic is pouring ...,A portal has opened and dark magic is pouring ...,A portal has opened and dark magic is pouring ...


In [24]:
steam_long_out.to_csv('steam_long.csv',index=False)
steam_wide_out.to_csv('steam_wide.csv',index=False)